In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

print(tf.__version__)

2.7.0-dev20210709


In [2]:
train_ds = pd.read_csv('./data/classification/train_dataset.csv')
test_ds  = pd.read_csv('./data/classification/test_dataset.csv')
valid_ds   = pd.read_csv('./data/classification/dev_dataset.csv')

In [3]:
train_ds.head()

,Sentence,Intent
0,Can you show me some movie listings?,INFORM_INTENT
1,I am in San Jose. Also I want to see a 3D movie.,INFORM
2,"Yes, please find me some other movies. I want ...",INFORM
3,What times do you have available for Curse of ...,INFORM_INTENT
4,March 5th.,INFORM


In [4]:
intents_list = []
intents_list.extend(train_ds['Intent'].unique())
intents_list.extend(test_ds['Intent'].unique())
intents_list.extend(valid_ds['Intent'].unique())
intents_list = set(intents_list)
intents_list

{'AFFIRM',
 'AFFIRM_INTENT',
 'INFORM',
 'INFORM_INTENT',
 'NEGATE',
 'NEGATE_INTENT',
 'REQUEST',
 'REQUEST_ALTS',
 'SELECT',
 'THANK_YOU'}

In [5]:
intent_mapper = {
'NEGATE_INTENT': 0,
'INFORM_INTENT': 1,
'AFFIRM': 2,
'SELECT': 3,
'AFFIRM_INTENT': 4,
'THANK_YOU': 5,
'REQUEST_ALTS': 6,
'INFORM': 7,
'REQUEST': 8,
'NEGATE': 9
}

In [6]:
train_ds['Intent'] = np.array(train_ds.Intent.map(lambda x: intent_mapper[x]))
test_ds['Intent'] = np.array(test_ds.Intent.map(lambda x: intent_mapper[x]))
valid_ds['Intent'] = np.array(valid_ds.Intent.map(lambda x: intent_mapper[x]))
train_ds.head()

,Sentence,Intent
0,Can you show me some movie listings?,1
1,I am in San Jose. Also I want to see a 3D movie.,7
2,"Yes, please find me some other movies. I want ...",7
3,What times do you have available for Curse of ...,1
4,March 5th.,7


In [7]:
train_ds['Intent'].unique()

array([1, 7, 3, 8, 0, 9, 6])

In [8]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [9]:
train_text = tf.data.Dataset.from_tensor_slices(train_ds['Sentence'])
train_labels = tf.data.Dataset.from_tensor_slices(train_ds['Intent'])

test_text = tf.data.Dataset.from_tensor_slices(test_ds['Sentence'])
test_labels = tf.data.Dataset.from_tensor_slices(test_ds['Intent'])

valid_text = tf.data.Dataset.from_tensor_slices(valid_ds['Sentence'])
valid_labels = tf.data.Dataset.from_tensor_slices(valid_ds['Intent'])


train_dataset = tf.data.Dataset.zip((train_text, train_labels))
test_dataset = tf.data.Dataset.zip((test_text, test_labels))
valid_dataset = tf.data.Dataset.zip((valid_text, valid_labels))

In [10]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
valid_dataset = valid_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [11]:
for example, label in train_dataset.take(1):
    print('texts: ', example.numpy()[:3])
    print()
    print('labels: ', label.numpy()[:3])

texts:  [b'No not right now' b'That sounds good. Thanks.'
 b"Great, that's all I need."]

labels:  [0 3 3]


# Text encoder

In [12]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [13]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'i', 'to', 'the', 'is', 'a', 'movie', 'watch', 'in',
       'what', 'for', 'it', 'you', 'movies', 'want', 'can', 'of', 'no',
       'like'], dtype='<U15')

In [14]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[18, 35, 56, 30,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [26, 27, 28, 36,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [40, 39, 31,  2, 53,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]])

In [15]:
for n in range(3):
    print("Original: ", example[n].numpy())
    print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
    print()

Original:  b'No not right now'
Round-trip:  no not right now                

Original:  b'That sounds good. Thanks.'
Round-trip:  that sounds good thanks                

Original:  b"Great, that's all I need."
Round-trip:  great thats all i need               



# create model

In [16]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='sigmoid'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])


In [17]:
# predict on a sample text without padding.

sample_text = ('Please check the showtimes for next Friday.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[-0.38046253]


In [18]:
# predict on a sample text with padding

padding = "the " * 2000

predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[-0.38046253]


In [19]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

# Train the model

In [23]:
history = model.fit(train_dataset, epochs=5,
                    validation_data=valid_dataset,
                    validation_steps=30)

Epoch 1/5
30/30 [==============================] - 3s 115ms/step - loss: 0.0000e+00 - accuracy: 0.1229 - val_loss: 0.0000e+00 - val_accuracy: 0.1385
Epoch 2/5
30/30 [==============================] - 4s 121ms/step - loss: 0.0000e+00 - accuracy: 0.1261 - val_loss: 0.0000e+00 - val_accuracy: 0.1349
Epoch 3/5
30/30 [==============================] - 3s 115ms/step - loss: 0.0000e+00 - accuracy: 0.1335 - val_loss: 0.0000e+00 - val_accuracy: 0.1344
Epoch 4/5
30/30 [==============================] - 4s 122ms/step - loss: 0.0000e+00 - accuracy: 0.1245 - val_loss: 0.0000e+00 - val_accuracy: 0.1339
Epoch 5/5
30/30 [==============================] - 3s 113ms/step - loss: 0.0000e+00 - accuracy: 0.1255 - val_loss: 0.0000e+00 - val_accuracy: 0.1333


In [24]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

41/41 [==============================] - 1s 34ms/step - loss: 0.0000e+00 - accuracy: 0.1213
Test Loss: 0.0
Test Accuracy: 0.1212700828909874


In [22]:
# predict on a sample text without padding.

sample_text = ('Please check the showtimes for next Friday.')
predictions = model.predict(np.array([sample_text]))
print(predictions)

[[1.4123327]]
